In [12]:
# Carga de librerias
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import glob
import os
from PIL import Image
from itertools import chain
from collections import Counter
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from keras import backend as K
from itertools import chain
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
# Definición de las variables globales
IMG_SIZE=32
NUM_CHANNELS=3
NUM_CLASSES=43
from random import sample, seed
seed(3)



In [14]:
# Declaración de los directorios
current_dir=os.getcwd()
dataset_dir=current_dir+"/Dataset"
dir_train=dataset_dir+"/Train"
train_data=pd.read_csv(dataset_dir+"/Train.csv")
test_data=pd.read_csv(dataset_dir+"/Test.csv")
meta_data=pd.read_csv(dataset_dir+"/Meta.csv")

x_images=test_data["Path"]
x_train_images=train_data["Path"]
x_train_labels=train_data[['ClassId']].to_numpy()
y_test=test_data[['ClassId']].values

In [15]:
# Preparación de los datos de entrenamiento
def train_data(size,shear,rotation,brig):
    # Definimos el data generator
    train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        validation_split=0.2,
        shear_range=shear,
        rotation_range=rotation,
        brightness_range=brig
    )

    # Generamos los datos de entrenamiento
    train_generator = train_datagen.flow_from_directory(
        directory=dir_train,
        target_size=size,
        class_mode="categorical",
        subset='training',
        shuffle=True
        seed=3)


    # Generamos los datos de vaidacion
    valid_generator = train_datagen.flow_from_directory(
        directory=dir_train,
        target_size=size,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
        seed=3)
    return train_generator,valid_generator

In [16]:
# Preparación de los datos de prueba
def test_generator(img_size, x_images):
    # Recorremos todas las imágenes
    for image_path in x_images:

        # Para cada imagen realizamos un resizing y lo convertimos en un array 
        image = load_img(dataset_dir+"/"+image_path, target_size=img_size)
        image_array = img_to_array(image)
        image_array=image_array/255
        image_array = np.array([image_array])
        
        # Pasamos la imagen a la función de preprocesado utilizada en el modelo
        yield image_array
        

In [17]:
# Generación de los pesos de las imágenes
def weights_train(train_data):
    # Contamos el numero de valores por clase
    counter = Counter(train_data.classes)
    # Almacenamos los valores máximos
    max_val = float(max(counter.values()))
    # Calculamos los pesos
    class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     
    return class_weights

In [8]:
# Configuraciones de los modelos preentrenados
settings = {
    'inceptionV3': {
        'model': InceptionV3,
        'img_size': (150, 150),
        'shear': 0.2,
        'brig': None,
        'rotation': None,
    },
    'efficientb0': {
        'model': EfficientNetB0,
        'img_size': (224, 224),
        'shear': 0.2,
        'brig': [0.8,0.8],
        'rotation': None
    },
    'resnet50': {
        'model': ResNet50,
        'img_size': (224, 224),
        'shear': 0.2,
        'brig': [0.8,0.8],
        'rotation': None    },
    'DenseNet121': {
        'model': DenseNet121,
        'img_size': (224, 224),
        'shear': 0.2,
        'brig': None,
        'rotation': None    }
}

In [30]:
# Función que crea el modelo y lo entrena
def imagenet_model (model, input_size, train, valid,  test=False ):
    weights=weights_train(train)
    base_model=model(include_top=False, weights="imagenet", input_shape=(input_size + (3,)))
    base_model_img=Sequential()
    base_model_img.add(base_model)
    base_model_img.add(Flatten())
    base_model_img.add(Dense(NUM_CLASSES, activation="softmax"))
    if test==False:
        epochs=15
    else:
        epochs=15
    # Compilamos el modelo
  
    base_model_img.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    # Generamos el callback de early stopping indicando que la validacion se realizara cada 50 epocas
    callback=EarlyStopping(monitor='val_loss', verbose=1, patience=6,restore_best_weights=True)
    # Entrenamos el modelo
    history = base_model_img.fit(train ,validation_data=(valid), 
                                                  epochs=epochs, verbose=1, callbacks=[callback], class_weight=weights)  
    
    return base_model_img, history

In [23]:
# Entrenamiento y predicción
def train_imagenet_models (settings):
    results=[]
    models=[]
    for name, model_settings in settings.items():
        model=model_settings['model']
        size=model_settings["img_size"]
        brig=model_settings["brig"]
        shear=model_settings["shear"]
        rotation=model_settings["rotation"]
        
        train,valid=train_data(size,shear,rotation,brig)
        cnn_model, history_model=imagenet_model(model,size,train, valid,test=False)
        
        X_test=test_generator(size,x_images)
        y_pred = cnn_model.predict(X_test)
        y_pred=np.argmax(y_pred,axis=1)
        
        label_mapper = {v: k for k, v in valid.class_indices.items()}
        y_pred = list(map(lambda x: label_mapper[x], y_pred))
        y_true=list(map(str,test_data["ClassId"]))
        val_acc=accuracy_score(y_pred,y_true)
        
        params=cnn_model.count_params()
        
        tmp={"Model": name,
         "Accuracy": val_acc,
         "Number of parameters": params
             }
        results.append(tmp)
        model_name=current_dir+'/Results/Pretrained_train/'+name+'.h5'
        cnn_model.save(model_name)
        
    return pd.DataFrame(results)


In [ ]:
result=train_imagenet_models(settings)
result.to_csv(current_dir+'/Results/Pretrained_train/CNN_Imagenet.csv', index=False)